In [67]:
!pip install human-eval

In [77]:
from human_eval.data import read_problems
sample_problem = read_problems()['HumanEval/84']

In [78]:
sample_problem['test']

'def check(candidate):\n\n    # Check some simple cases\n    assert True, "This prints if this assert fails 1 (good for debugging!)"\n    assert candidate(1000) == "1", "Error"\n    assert candidate(150) == "110", "Error"\n    assert candidate(147) == "1100", "Error"\n\n    # Check some edge cases that are easy to work out by hand.\n    assert True, "This prints if this assert fails 2 (also good for debugging!)"\n    assert candidate(333) == "1001", "Error"\n    assert candidate(963) == "10010", "Error"\n\n'

In [84]:
def solve(N):
    """Given a positive integer N, return the total sum of its decimal digits in binary.

    Example:
        For N = 1000, the sum of digits will be 1, the output should be "1".
        For N = 150, the sum of digits will be 6, the output should be "110".
        For N = 147, the sum of digits will be 12, the output should be "1100".

    Variables:
        @N integer
             Constraints: 0 ≤ N ≤ 10000.
    Output:
         a string of binary number
    """
    digit_sum = sum(int(digit) for digit in str(N))  # Sum of decimal digits
    return bin(digit_sum)[2:]  # Convert sum to binary and remove '0b'
solve(147)

'1100'

In [26]:
print(sample_problem)
print("test")
print(sample_problem['test'])
print("*"*100)
import pandas as pd

# Path to your Excel file
file_path = "/content/planning_gemma.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Display the DataFram
print("/n/n")
print("code ")
print(df['extracted_code'][0])


{'task_id': 'HumanEval/1', 'prompt': 'from typing import List\n\n\ndef separate_paren_groups(paren_string: str) -> List[str]:\n    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to\n    separate those group into separate strings and return the list of those.\n    Separate groups are balanced (each open brace is properly closed) and not nested within each other\n    Ignore any spaces in the input string.\n    >>> separate_paren_groups(\'( ) (( )) (( )( ))\')\n    [\'()\', \'(())\', \'(()())\']\n    """\n', 'entry_point': 'separate_paren_groups', 'canonical_solution': "    result = []\n    current_string = []\n    current_depth = 0\n\n    for c in paren_string:\n        if c == '(':\n            current_depth += 1\n            current_string.append(c)\n        elif c == ')':\n            current_depth -= 1\n            current_string.append(c)\n\n            if current_depth == 0:\n                result.append(''.join(current_string

{'error': "Execution failed: type complex doesn't define __round__ method",
 'traceback': 'Traceback (most recent call last):\n  File "/tmp/ipython-input-2309236678.py", line 35, in evaluate_generated_code\n    check_func(candidate_func)\n  File "<string>", line 6, in check\n  File "<string>", line 16, in iscube\nTypeError: type complex doesn\'t define __round__ method\n'}

In [66]:
output_path = "/content/eval_results_gemma.xlsx"  # change path as needed
new_df.to_excel(output_path, index=False)



In [59]:
import traceback
import ast

def evaluate_generated_code(problem_dict, generated_code: str):
    """
    Evaluate generated code against HumanEval-style tests.
    Always runs *all* asserts individually, capturing input, expected, actual outputs.

    Args:
        problem_dict (dict): HumanEval-like dict with keys:
            - 'entry_point'
            - 'test'
        generated_code (str): Model-generated code string

    Returns:
        dict: {'results': list of per-test dictionaries}
    """
    entry_point = problem_dict['entry_point']
    test_code = problem_dict['test']
    results = []

    try:
        # Compile and load generated code
        namespace = {}
        exec(generated_code, namespace)
        if entry_point not in namespace:
            return {"error": f"Entry point '{entry_point}' not found in generated code."}
        candidate_func = namespace[entry_point]
    except Exception as e:
        return {"error": f"Failed to execute generated code: {e}", "traceback": traceback.format_exc()}

    # Extract assert statements only
    test_lines = [l.strip() for l in test_code.splitlines() if l.strip().startswith("assert ")]
    if not test_lines:
        return {"error": "No assert statements found in test code."}

    for line in test_lines:
        try:
            expr = line.replace("assert ", "").strip()
            if "==" not in expr:
                continue

            left, right = expr.split("==", 1)
            left = left.strip()
            right = right.strip()

            # Extract the candidate call and its arguments
            input_part = left[left.find("(")+1:left.rfind(")")]
            expected_output = ast.literal_eval(right)

            # Evaluate actual output safely
            try:
                actual_output = eval(f"candidate({input_part})", {"candidate": candidate_func})
                passed = actual_output == expected_output
                results.append({
                    "test_case": expr,
                    "input": input_part,
                    "expected": expected_output,
                    "actual": actual_output,
                    "status": "✅ Passed" if passed else "❌ Failed"
                })
            except Exception as e:
                results.append({
                    "test_case": expr,
                    "input": input_part,
                    "expected": expected_output,
                    "actual": str(e),
                    "status": f"❌ Runtime Error ({type(e).__name__})"
                })

        except Exception as e:
            results.append({
                "test_case": line,
                "error": f"Failed to parse test line: {e}",
                "status": "⚠️ Parse Error"
            })

    return {"results": results}
generated_code = df['extracted_code'][21]  # or any generated code
sample_problem = read_problems()[df['task_id'][21]]

results = evaluate_generated_code(sample_problem, generated_code)


In [60]:
results

{'results': [{'test_case': 'candidate(123) == (8, 13)',
   'input': '123',
   'expected': (8, 13),
   'actual': (8, 13),
   'status': '✅ Passed'},
  {'test_case': 'candidate(12) == (4, 6)',
   'input': '12',
   'expected': (4, 6),
   'actual': (4, 6),
   'status': '✅ Passed'},
  {'test_case': 'candidate(3) == (1, 2)',
   'input': '3',
   'expected': (1, 2),
   'actual': (1, 2),
   'status': '✅ Passed'},
  {'test_case': 'candidate(63) == (6, 8)',
   'input': '63',
   'expected': (6, 8),
   'actual': (6, 8),
   'status': '✅ Passed'},
  {'test_case': 'candidate(25) == (5, 6)',
   'input': '25',
   'expected': (5, 6),
   'actual': (5, 6),
   'status': '✅ Passed'},
  {'test_case': 'candidate(19) == (4, 6)',
   'input': '19',
   'expected': (4, 6),
   'actual': (4, 6),
   'status': '✅ Passed'},
  {'test_case': 'candidate(9) == (4, 5), "This prints if this fails 1 (good for debugging!)"',
   'input': '9',
   'expected': ((4, 5), 'This prints if this fails 1 (good for debugging!)'),
   'actual

In [38]:
new_df=df.copy()
new_df['output_description']=['']*len(df)
for ind in df.index:
  sample_problem = read_problems()[df['task_id'][ind]]

  new_df['output_description'][ind]=evaluate_generated_code(sample_problem,df['extracted_code'][ind])


/tmp/ipython-input-1421575472.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  new_df['output_description'][ind]=evaluate_generated_code(sample_problem,df['extracted_code'][ind])
/tmp/ipython-input-1421575472.py:6: SettingWithCopyWarning:

In [39]:
new_df

,task_id,prompt,model_output_index,model_output,extracted_code,result,passed,pass@1,pass@3,output_description
0,HumanEval/1,You are an expert Python programmer. Given the...,1,```python\nfrom typing import List\n\n\ndef se...,from typing import List\n\n\ndef separate_pare...,"{""task_id"": ""HumanEval/1"", ""passed"": true, ""re...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
1,HumanEval/1,You are an expert Python programmer. Given the...,2,```python\nfrom typing import List\n\n\ndef se...,from typing import List\n\n\ndef separate_pare...,"{""task_id"": ""HumanEval/1"", ""passed"": true, ""re...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
2,HumanEval/1,You are an expert Python programmer. Given the...,3,```python\nfrom typing import List\n\n\ndef se...,from typing import List\n\n\ndef separate_pare...,"{""task_id"": ""HumanEval/1"", ""passed"": true, ""re...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
3,HumanEval/17,You are an expert Python programmer. Given the...,1,```python\nfrom typing import List\n\n\ndef pa...,from typing import List\n\n\ndef parse_music(m...,"{""task_id"": ""HumanEval/17"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
4,HumanEval/17,You are an expert Python programmer. Given the...,2,```python\nfrom typing import List\n\n\ndef pa...,from typing import List\n\n\ndef parse_music(m...,"{""task_id"": ""HumanEval/17"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
5,HumanEval/17,You are an expert Python programmer. Given the...,3,```python\nfrom typing import List\n\n\ndef pa...,from typing import List\n\n\ndef parse_music(m...,"{""task_id"": ""HumanEval/17"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
6,HumanEval/26,You are an expert Python programmer. Given the...,1,```python\nfrom typing import List\n\n\ndef re...,from typing import List\n\n\ndef remove_duplic...,"{""task_id"": ""HumanEval/26"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
7,HumanEval/26,You are an expert Python programmer. Given the...,2,```python\nfrom typing import List\n\n\ndef re...,from typing import List\n\n\ndef remove_duplic...,"{""task_id"": ""HumanEval/26"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
8,HumanEval/26,You are an expert Python programmer. Given the...,3,```python\nfrom typing import List\n\n\ndef re...,from typing import List\n\n\ndef remove_duplic...,"{""task_id"": ""HumanEval/26"", ""passed"": true, ""r...",True,1,1,"{'results': [('All tests', '✅ Passed')]}"
9,HumanEval/77,You are an expert Python programmer. Given the...,1,```python\ndef iscube(a):\n '''\n Write ...,def iscube(a):\n '''\n Write a function ...,"{""task_id"": ""HumanEval/77"", ""passed"": false, ""...",False,0,0,{'error': 'Execution failed: type complex does...


In [29]:
print(df['extracted_code'][11])

def iscube(a):
    '''
    Write a function that takes an integer a and returns True 
    if this ingeger is a cube of some integer number.
    Note: you may assume the input is always valid.
    Examples:
    iscube(1) ==> True
    iscube(2) ==> False
    iscube(-1) ==> True
    iscube(64) ==> True
    iscube(0) ==> True
    iscube(180) ==> False
    '''
    # Calculate the cube root of the input number.
    # If the cube of the cube root is equal to the original number, then it is a cube.
    cube_root = round(a**(1/3))
    return cube_root**3 == a


In [12]:
#!/usr/bin/env python3
"""
Detailed Test Analyzer - Enhanced version of _run_test_cases
Captures individual test case results and detailed error information
"""
from human_eval.data import read_problems
sample_problem = read_problems()['HumanEval/1']


import os
import subprocess
import tempfile
import re
from datetime import datetime
from typing import Dict, List, Any

def run_detailed_test_analysis(function_code: str, problem: Dict, model_provider: str = "unknown") -> Dict:
    """
    Enhanced test runner that captures individual test case results
    """

    try:
        # Get test cases from problem
        test_cases = problem.get('test', '').strip()
        if not test_cases:
            return create_error_result('No test cases available')

        # Extract individual assert statements
        assert_lines = extract_assert_statements(test_cases)

        if not assert_lines:
            return create_error_result('No assert statements found')

        # Create enhanced test file with individual test tracking
        test_content = create_enhanced_test_file(function_code, assert_lines, problem)

        # Write and execute test
        with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False, encoding='utf-8') as f:
            f.write(test_content)
            temp_file = f.name

        try:
            # Execute the test
            result = subprocess.run(
                ['python', temp_file],
                capture_output=True,
                text=True,
                timeout=15
            )

            # Parse detailed results
            detailed_results = parse_detailed_output(result.stdout, result.stderr, assert_lines)

            return detailed_results

        finally:
            # Clean up
            if os.path.exists(temp_file):
                os.remove(temp_file)

    except subprocess.TimeoutExpired:
        return create_timeout_result(len(assert_lines) if 'assert_lines' in locals() else 0)
    except Exception as e:
        return create_error_result(f"Test execution failed: {str(e)}")

def extract_assert_statements(test_cases: str) -> List[str]:
    """Extract individual assert statements from test cases"""

    assert_lines = []
    for line in test_cases.split('\n'):
        line = line.strip()
        if line.startswith('assert '):
            assert_lines.append(line)

    return assert_lines

def create_enhanced_test_file(function_code: str, assert_lines: List[str], problem: Dict) -> str:
    """Create test file that runs each assertion individually and reports detailed results"""

    function_name = extract_function_name(function_code)
    individual_tests = []

    for i, assert_line in enumerate(assert_lines, 1):
        test_call, expected = parse_assertion(assert_line)

        # Each test is isolated in its own function
        individual_test = f'''
def test_{i}():
    """Test case {i}: {assert_line}"""
    try:
        actual = eval({repr(test_call.replace('candidate', function_name))})
        expected = eval({repr(expected)})
        if actual == expected:
            print(f"[PASS] Test {i}: PASS - {test_call} == {{expected}}")
            return True
        else:
            print(f"[FAIL] Test {i}: FAIL - {test_call}")
            print(f"   Expected: {{expected}}")
            print(f"   Actual: {{actual}}")
            return False
    except Exception as e:
        print(f"[ERROR] Test {i}: ERROR - {test_call}")
        print(f"   Error: {{str(e)}}")
        print(f"   Type: {{type(e).__name__}}")
        return False
'''
        individual_tests.append(individual_test)

    # Combine everything into one executable test script
    test_content = f'''
import sys
import traceback
from typing import List, Any

{function_code}

{chr(10).join(individual_tests)}

def main():
    """Run all tests individually and report results"""

    print("DETAILED TEST EXECUTION")
    print("=" * 50)

    results = []
    passed_count = 0

    # Run each test individually
    for i in range(1, {len(assert_lines) + 1}):
        test_func = globals()[f'test_{{i}}']
        try:
            success = test_func()
            results.append(success)
            if success:
                passed_count += 1
        except Exception as e:
            print(f"[CRITICAL] Test {{i}}: CRITICAL ERROR - {{str(e)}}")
            results.append(False)

    print("=" * 50)
    print(f"SUMMARY: {{passed_count}}/{len(assert_lines)} tests passed")

    # Exit with appropriate code
    sys.exit(0 if passed_count == {len(assert_lines)} else 1)

if __name__ == "__main__":
    main()
'''

    return test_content


# def create_enhanced_test_file(function_code: str, assert_lines: List[str], problem: Dict) -> str:
#     """Create test file that runs each assertion individually"""

#     function_name = extract_function_name(function_code)

#     # Create individual test functions
#     individual_tests = []
#     for i, assert_line in enumerate(assert_lines, 1):
#         # Parse assertion: assert candidate([1.0, 2.0], 0.5) == False
#         test_call, expected = parse_assertion(assert_line)

#         individual_test = f'''
# def test_{i}():
#     """Test case {i}: {assert_line}"""
#     try:
#         actual = {test_call.replace('candidate', function_name)}
#         expected = {expected}

#         if actual == expected:
#             print(f"[PASS] Test {i}: PASS - {test_call} == {{expected}}")
#             return True
#         else:
#             print(f"[FAIL] Test {i}: FAIL - {test_call}")
#             print(f"   Expected: {{expected}}")
#             print(f"   Actual: {{actual}}")
#             return False

#     except Exception as e:
#         print(f"[ERROR] Test {i}: ERROR - {test_call}")
#         print(f"   Error: {{str(e)}}")
#         print(f"   Type: {{type(e).__name__}}")
#         return False
# '''
#         individual_tests.append(individual_test)

#     # Create main test runner
#     test_content = f'''
# import sys
# import traceback
# from typing import List, Any

# {function_code}

# {chr(10).join(individual_tests)}

# def main():
#     """Run all tests individually and report results"""

#     print("DETAILED TEST EXECUTION")
#     print("=" * 50)

#     results = []
#     passed_count = 0

#     # Run each test individually
#     for i in range(1, {len(assert_lines) + 1}):
#         test_func = globals()[f'test_{{i}}']
#         try:
#             success = test_func()
#             results.append(success)
#             if success:
#                 passed_count += 1
#         except Exception as e:
#             print(f"[CRITICAL] Test {{i}}: CRITICAL ERROR - {{str(e)}}")
#             results.append(False)

#     print("=" * 50)
#     print(f"SUMMARY: {{passed_count}}/{len(assert_lines)} tests passed")

#     # Exit with appropriate code
#     sys.exit(0 if passed_count == {len(assert_lines)} else 1)

# if __name__ == "__main__":
#     main()
# '''

#     return test_content

def extract_function_name(function_code: str) -> str:
    """Extract function name from code"""
    match = re.search(r'def (\w+)', function_code)
    return match.group(1) if match else 'unknown_function'

def parse_assertion(assert_line: str) -> tuple:
    """Parse assertion into test call and expected result"""

    # Remove 'assert ' prefix
    assertion = assert_line.replace('assert ', '').strip()

    # Split on comparison operators
    if ' == ' in assertion:
        test_call, expected = assertion.split(' == ', 1)
        return test_call.strip(), expected.strip()
    elif ' != ' in assertion:
        test_call, expected = assertion.split(' != ', 1)
        return test_call.strip(), f"not ({expected.strip()})"
    elif ' > ' in assertion:
        test_call, expected = assertion.split(' > ', 1)
        return f"({test_call.strip()}) > ({expected.strip()})", "True"
    elif ' < ' in assertion:
        test_call, expected = assertion.split(' < ', 1)
        return f"({test_call.strip()}) < ({expected.strip()})", "True"
    else:
        # Simple assertion without comparison
        return assertion, "True"

def parse_detailed_output(stdout: str, stderr: str, assert_lines: List[str]) -> Dict:
    """Parse detailed output from enhanced test execution"""

    lines = stdout.split('\n')
    test_results = []
    passed_count = 0

    # Parse individual test results
    for line in lines:
        if line.startswith('[PASS] Test '):
            # Parse: [PASS] Test 1: PASS - has_close_elements([1.0, 2.0, 3.0], 0.5) == False
            parts = line.split(': ', 2)
            if len(parts) >= 3:
                test_num = int(parts[0].split()[-1])
                status = parts[1]
                description = parts[2] if len(parts) > 2 else ""

                test_results.append({
                    'test_number': test_num,
                    'status': 'PASS',
                    'description': description,
                    'expected': None,
                    'actual': None,
                    'error': None
                })
                passed_count += 1

        elif line.startswith('[FAIL] Test '):
            # Parse failure
            test_num = int(line.split()[2].rstrip(':'))
            description = line.split(': ', 2)[2] if ': ' in line else ""

            # Look for expected/actual values in next lines
            expected = None
            actual = None

            test_results.append({
                'test_number': test_num,
                'status': 'FAIL',
                'description': description,
                'expected': expected,
                'actual': actual,
                'error': 'Assertion failed'
            })

        elif line.startswith('[ERROR] Test '):
            # Parse error
            test_num = int(line.split()[2].rstrip(':'))
            description = line.split(': ', 2)[2] if ': ' in line else ""

            test_results.append({
                'test_number': test_num,
                'status': 'ERROR',
                'description': description,
                'expected': None,
                'actual': None,
                'error': 'Runtime error'
            })

    # If no detailed results found, fall back to basic analysis
    if not test_results:
        total_tests = len(assert_lines)
        all_passed = "All tests passed!" in stdout or stderr == ""
        passed_tests = total_tests if all_passed else 0

        return {
            'all_passed': all_passed,
            'total_test_cases': total_tests,
            'passed_test_cases': passed_tests,
            'test_results': [],
            'output': f"{passed_tests}/{total_tests} tests passed",
            'error': stderr.strip() if stderr else None
        }

    return {
        'all_passed': passed_count == len(assert_lines),
        'total_test_cases': len(assert_lines),
        'passed_test_cases': passed_count,
        'test_results': test_results,
        'output': f"{passed_count}/{len(assert_lines)} tests passed",
        'error': stderr.strip() if stderr else None
    }

def create_error_result(error_msg: str) -> Dict:
    """Create error result structure"""
    return {
        'all_passed': False,
        'total_test_cases': 0,
        'passed_test_cases': 0,
        'test_results': [],
        'output': '0/0 tests passed (error)',
        'error': error_msg
    }

def create_timeout_result(total_tests: int) -> Dict:
    """Create timeout result structure"""
    return {
        'all_passed': False,
        'total_test_cases': total_tests,
        'passed_test_cases': 0,
        'test_results': [],
        'output': f'0/{total_tests} tests passed (timeout)',
        'error': 'Test execution timed out'
    }

# Example usage
if __name__ == "__main__":
    # Test the enhanced analyzer
    sample_function = '''
# The problem requires us to separate groups of nested parentheses in a given string into separate strings.
# We will use a stack-based approach to keep track of the opening and closing parentheses and form the groups.
# The main steps include removing spaces from the input string, iterating over the string to identify the groups, and appending them to the result list.

from typing import List

def separate_paren_groups(paren_string: str) -> List[str]:
    """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
    separate those group into separate strings and return the list of those.
    Separate groups are balanced (each open brace is properly closed) and not nested within each other
    Ignore any spaces in the input string.
    >>> separate_paren_groups('( ) (( )) (( )( ))')
    ['()', '(())', '(()())']
    """
    paren_string = paren_string.replace(" ", "")
    result = []
    current_group = ""
    balance = 0
    for char in paren_string:
        current_group += char
        if char == "(":
            balance += 1
        elif char == ")":
            balance -= 1
        if balance == 0:
            result.append(current_group)
            current_group = ""
    return result

# Test the function
print(separate_paren_groups('( ) (( )) (( )( ))'))  # Output: ('()', '(())', '(()())')
'''

    sample_problem = read_problems()['HumanEval/1']


    print("TESTING ENHANCED TEST ANALYZER")
    print("=" * 50)

    result = run_detailed_test_analysis(sample_function, sample_problem, "test")

    print(f"\nResults:")
    print(f"Overall: {result['output']}")
    print(f"All Passed: {result['all_passed']}")

    if result['test_results']:
        print(f"\nIndividual Test Results:")
        for test in result['test_results']:
            status_icon = "[PASS]" if test['status'] == 'PASS' else "[FAIL]" if test['status'] == 'FAIL' else "[ERROR]"
            print(f"  {status_icon} Test {test['test_number']}: {test['status']}")
            print(f"      {test['description']}")
            if test['error']:
                print(f"      Error: {test['error']}")

    if result['error']:
        print(f"\nError: {result['error']}")

TESTING ENHANCED TEST ANALYZER

Results:
Overall: 0/0 tests passed (error)
All Passed: False

Error: Test execution failed: list index out of range


In [15]:
#!/usr/bin/env python3
"""
Enhanced Test Analyzer for HumanEval
Captures individual test case results and detailed error information
"""

import os
import re
import subprocess
import tempfile
from typing import List, Dict, Any

# ============================================================
# 1. ASSERT EXTRACTION
# ============================================================

def extract_asserts(test_code: str, entry_point: str) -> List[str]:
    """
    Extracts all assert lines from HumanEval test code.
    Works even if asserts are indented inside a function.
    """
    pattern = re.compile(r"^\s*assert\s+.*")
    lines = [line.strip() for line in test_code.splitlines() if pattern.match(line)]
    return lines


# ============================================================
# 2. HELPER: FUNCTION NAME EXTRACTION
# ============================================================

def extract_function_name(function_code: str) -> str:
    """Extracts the function name from the generated solution."""
    match = re.search(r"def\s+(\w+)\s*\(", function_code)
    return match.group(1) if match else "candidate"


# ============================================================
# 3. HELPER: ASSERT PARSER
# ============================================================

def parse_assertion(assert_line: str):
    """
    Parse an assert line like `assert candidate(1, 2) == 3`
    into (call_expr, expected_expr)
    """
    try:
        left, right = assert_line.replace("assert ", "").split("==")
        return left.strip(), right.strip()
    except Exception:
        return assert_line, "None"


# ============================================================
# 4. MAIN TEST FILE CREATOR
# ============================================================

def create_enhanced_test_file(function_code: str, assert_lines: List[str], problem: Dict) -> str:
    """Create test file that runs each assertion individually and reports detailed results"""

    function_name = extract_function_name(function_code)
    individual_tests = []

    for i, assert_line in enumerate(assert_lines, 1):
        test_call, expected = parse_assertion(assert_line)

        individual_test = f'''
def test_{i}():
    """Test case {i}: {assert_line}"""
    try:
        actual = eval({repr(test_call.replace('candidate', function_name))})
        expected = eval({repr(expected)})
        if actual == expected:
            print(f"[PASS] Test {i}: PASS - {test_call} == {{expected}}")
            return True
        else:
            print(f"[FAIL] Test {i}: FAIL - {test_call}")
            print(f"   Expected: {{expected}}")
            print(f"   Actual: {{actual}}")
            return False
    except Exception as e:
        print(f"[ERROR] Test {i}: ERROR - {test_call}")
        print(f"   Error: {{str(e)}}")
        print(f"   Type: {{type(e).__name__}}")
        return False
'''
        individual_tests.append(individual_test)

    # Full executable test runner
    test_content = f'''
import sys
import traceback
from typing import List, Any

{function_code}

{chr(10).join(individual_tests)}

def main():
    """Run all tests individually and report results"""

    print("DETAILED TEST EXECUTION")
    print("=" * 50)

    results = []
    passed_count = 0

    # Run each test individually
    for i in range(1, {len(assert_lines) + 1}):
        test_func = globals()[f'test_{{i}}']
        try:
            success = test_func()
            results.append(success)
            if success:
                passed_count += 1
        except Exception as e:
            print(f"[CRITICAL] Test {{i}}: CRITICAL ERROR - {{str(e)}}")
            results.append(False)

    print("=" * 50)
    print(f"SUMMARY: {{passed_count}}/{len(assert_lines)} tests passed")
    sys.exit(0 if passed_count == {len(assert_lines)} else 1)

if __name__ == "__main__":
    main()
'''

    return test_content


# ============================================================
# 5. TEST EXECUTION WRAPPER
# ============================================================

def run_detailed_test(function_code: str, test_code: str, problem: Dict) -> Dict[str, Any]:
    """Run enhanced tests and capture detailed results."""
    assert_lines = extract_asserts(test_code, problem.get("entry_point", "candidate"))

    if not assert_lines:
        print("No asserts found — skipping test execution.")
        return {
            "passed": 0,
            "total": 0,
            "error": "No assert statements extracted",
            "all_passed": False
        }

    test_content = create_enhanced_test_file(function_code, assert_lines, problem)

    # Run in a temp file
    with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as f:
        f.write(test_content)
        temp_file = f.name

    try:
        result = subprocess.run(
            ["python3", temp_file],
            capture_output=True,
            text=True,
            timeout=20
        )
        output = result.stdout + result.stderr
        success = (result.returncode == 0)
    except subprocess.TimeoutExpired:
        output = "Timeout expired while running tests."
        success = False
    except Exception as e:
        output = f"Error during subprocess execution: {e}"
        success = False
    finally:
        os.remove(temp_file)

    # Parse pass counts from the output
    passed = len(re.findall(r"\[PASS\]", output))
    total = len(assert_lines)

    print("TESTING ENHANCED TEST ANALYZER")
    print("=" * 50)
    print(output)
    print("=" * 50)
    print(f"Results:\nOverall: {passed}/{total} tests passed")
    print(f"All Passed: {success}")

    return {
        "passed": passed,
        "total": total,
        "output": output,
        "all_passed": success
    }


# ============================================================
# 6. QUICK DEMO
# ============================================================

if __name__ == "__main__":
    # Example demo using a HumanEval-like problem
    #

    sample_function = '''
    # The problem requires us to separate groups of nested parentheses in a given string into separate strings.
    # We will use a stack-based approach to keep track of the opening and closing parentheses and form the groups.
    # The main steps include removing spaces from the input string, iterating over the string to identify the groups, and appending them to the result list.

    from typing import List

    def separate_paren_groups(paren_string: str) -> List[str]:
        """ Input to this function is a string containing multiple groups of nested parentheses. Your goal is to
        separate those group into separate strings and return the list of those.
        Separate groups are balanced (each open brace is properly closed) and not nested within each other
        Ignore any spaces in the input string.
        >>> separate_paren_groups('( ) (( )) (( )( ))')
        ['()', '(())', '(()())']
        """
        paren_string = paren_string.replace(" ", "")
        result = []
        current_group = ""
        balance = 0
        for char in paren_string:
            current_group += char
            if char == "(":
                balance += 1
            elif char == ")":
                balance -= 1
            if balance == 0:
                result.append(current_group)
                current_group = ""
        return result

    # Test the function
    print(separate_paren_groups('( ) (( )) (( )( ))'))  # Output: ('()', '(())', '(()())')
    '''

    sample_problem = read_problems()['HumanEval/1']

    run_detailed_test(sample_function, sample_problem["test"], sample_problem)


TESTING ENHANCED TEST ANALYZER
  File "/tmp/tmpszmb5oxh.py", line 11
    from typing import List
IndentationError: unexpected indent

Results:
Overall: 0/4 tests passed
All Passed: False
